# ChatGPT Over Databases

You can put ChatGPT over a database by using ChatGPT to convert natural-language questions such as "How many employees does Northwind have?" into SQL queries and then executing those queries against the database. The following examples use ChatGPT to expose information in Microsoft's Northwind database. They require [SQLite3](https://www.sqlite.org/index.html). Before running this notebook, create an environment variable named `API_KEY` and set it equal to your OpenAI API key.

Begin by using SQLite3 to connect to the database and list the names of all Northwind employees:

In [1]:
import sqlite3

connection = sqlite3.connect('Data/northwind.db')
cursor = connection.cursor()
cursor.execute('SELECT * FROM Employees')
rows = cursor.fetchall()

for row in rows:
    print(row[2] + ' ' + row[1])

Nancy Davolio
Andrew Fuller
Janet Leverling
Margaret Peacock
Steven Buchanan
Michael Suyama
Robert King
Laura Callahan
Anne Dodsworth


Now use ChatGPT to convert a question into a SQL query. Observe that the prompt includes a CREATE TABLE statement that details the structure of the Employees table to the LLM:

In [2]:
import os
from openai import OpenAI

client = OpenAI(api_key=os.environ['API_KEY'])
question = 'How many employees does Northwind have?'

prompt = f'''
Assume the database has a table named Employees that is defined as follows:

CREATE TABLE [Employees](
  [EmployeeID] INTEGER PRIMARY KEY AUTOINCREMENT,
  [LastName] TEXT,
  [FirstName] TEXT,
  [Title] TEXT,
  [TitleOfCourtesy] TEXT,
  [BirthDate] DATE,
  [HireDate] DATE,
  [Address] TEXT,
  [City] TEXT,
  [Region] TEXT,
  [PostalCode] TEXT,
  [Country] TEXT,
  [HomePhone] TEXT,
  [Extension] TEXT,
  [Photo] BLOB,
  [Notes] TEXT,
  [ReportsTo] INTEGER,
  [PhotoPath] TEXT
);

Generate a well-formed SQL query from the following prompt:

PROMPT: {question}
'''

messages = [
    { 'role': 'user', 'content': prompt }
]

response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=messages,
    temperature=0
)

sql = response.choices[0].message.content
print(sql)

SELECT COUNT(*) FROM Employees;


Execute the query against the database:

In [3]:
cursor.execute(sql)
result = cursor.fetchall()
print(result)

[(9,)]


Pass the question and the query results to ChatGPT and ask it to phrase an answer in natural language:

In [4]:
prompt = f'''
Given the following question and query result, phrase the answer
in terms that a human can understand.

QUESTION: {question}

RESULT: {result}
'''

messages = [
    { 'role': 'user', 'content': prompt }
]

response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=messages
)

print(response.choices[0].message.content)

Northwind has 9 employees.


Now let's try a question that involves the Products table:

In [5]:
question = 'Which products have been discontinued but are currently in stock?'

prompt = f'''
Assume the database has a table named Products that is defined as follows:

CREATE TABLE [Products](
  [ProductID]INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  [ProductName]TEXT NOT NULL,
  [SupplierID]INTEGER,
  [CategoryID]INTEGER,
  [QuantityPerUnit]TEXT,
  [UnitPrice]NUMERIC DEFAULT 0,
  [UnitsInStock]INTEGER DEFAULT 0,
  [UnitsOnOrder]INTEGER DEFAULT 0,
  [ReorderLevel]INTEGER DEFAULT 0,
  [Discontinued]TEXT NOT NULL DEFAULT '0'
);

Generate a well-formed SQL query from the following prompt:

PROMPT: {question}
'''

messages = [
    { 'role': 'user', 'content': prompt }
]

response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=messages,
    temperature=0
)

sql = response.choices[0].message.content
print(sql)

SELECT ProductName
FROM Products
WHERE Discontinued = '1' AND UnitsInStock > 0;


Execute the query against the database:

In [6]:
cursor.execute(sql)
result = cursor.fetchall()
print(result)

[('Mishi Kobe Niku',), ('Guaraná Fantástica',), ('Rössle Sauerkraut',), ('Singaporean Hokkien Fried Mee',)]


Ask ChatGPT the phrase the answer using natural language:

In [7]:
prompt = f'''
Given the following question and query result, phrase the answer
in terms that a human can understand.

QUESTION: {question}

RESULT: {result}
'''

messages = [
    { 'role': 'user', 'content': prompt }
]

response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=messages
)

print(response.choices[0].message.content)

The products that have been discontinued but are currently in stock are Mishi Kobe Niku, Guaraná Fantástica, Rössle Sauerkraut, and Singaporean Hokkien Fried Mee.


Do it again, but this time ask ChatGPT to include the number of units that are in stock for each item that has been discontinued as well as the combined value of all the units:

In [8]:
question = '''
Which products have been discontinued but are currently in stock,
how many of each is in stock, and what is the combined value of all
the products?
'''

prompt = f'''
Assume the database has a table named Products that is defined as follows:

CREATE TABLE [Products](
  [ProductID]INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  [ProductName]TEXT NOT NULL,
  [SupplierID]INTEGER,
  [CategoryID]INTEGER,
  [QuantityPerUnit]TEXT,
  [UnitPrice]NUMERIC DEFAULT 0,
  [UnitsInStock]INTEGER DEFAULT 0,
  [UnitsOnOrder]INTEGER DEFAULT 0,
  [ReorderLevel]INTEGER DEFAULT 0,
  [Discontinued]TEXT NOT NULL DEFAULT '0'
);

Generate a well-formed SQL query from the following prompt:

PROMPT: {question}
'''

messages = [
    { 'role': 'user', 'content': prompt }
]

response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=messages,
    temperature=0
)

sql = response.choices[0].message.content
print(sql)

SELECT ProductName, UnitsInStock, UnitPrice * UnitsInStock AS TotalValue
FROM Products
WHERE Discontinued = '1' AND UnitsInStock > 0;


In [9]:
cursor.execute(sql)
result = cursor.fetchall()
print(result)

[('Mishi Kobe Niku', 29, 2813), ('Guaraná Fantástica', 20, 90.0), ('Rössle Sauerkraut', 26, 1185.6000000000001), ('Singaporean Hokkien Fried Mee', 26, 364)]


In [10]:
prompt = f'''
Given the following question and query result, phrase the answer
in terms that a human can understand.

QUESTION: {question}

RESULT: {result}
'''

messages = [
    { 'role': 'user', 'content': prompt }
]

response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=messages
)

print(response.choices[0].message.content)

The products that have been discontinued but are currently in stock are Mishi Kobe Niku, Guaraná Fantástica, Rössle Sauerkraut, and Singaporean Hokkien Fried Mee. There are 29 units of Mishi Kobe Niku, 20 units of Guaraná Fantástica, 26 units of Rössle Sauerkraut, and 26 units of Singaporean Hokkien Fried Mee in stock. The combined value of all these products is $4,452.60.


Now try something more ambitious: a question that involves two tables. Note that the definition of the Products table now includes a foreign key relationship to the Suppliers table:

In [11]:
question = 'List all products that are currently out of stock and the suppliers for those products.'

prompt = f'''
Assume the database has table named Products and Suppliers that are defined as follows:

CREATE TABLE [Products](
  [ProductID]INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  [ProductName]TEXT NOT NULL,
  [SupplierID]INTEGER,
  [CategoryID]INTEGER,
  [QuantityPerUnit]TEXT,
  [UnitPrice]NUMERIC DEFAULT 0,
  [UnitsInStock]INTEGER DEFAULT 0,
  [UnitsOnOrder]INTEGER DEFAULT 0,
  [ReorderLevel]INTEGER DEFAULT 0,
  [Discontinued]TEXT NOT NULL DEFAULT '0',
  FOREIGN KEY ([SupplierID]) REFERENCES [Suppliers] ([SupplierID])
);

CREATE TABLE [Suppliers](
   [SupplierID]INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
   [CompanyName]TEXT NOT NULL,
   [ContactName]TEXT,
   [ContactTitle]TEXT,
   [Address]TEXT,
   [City]TEXT,
   [Region]TEXT,
   [PostalCode]TEXT,
   [Country]TEXT,
   [Phone]TEXT,
   [Fax]TEXT,
   [HomePage]TEXT
);

Generate a well-formed SQL query from the following prompt:

PROMPT: {question}
'''

messages = [
    { 'role': 'user', 'content': prompt }
]

response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=messages,
    temperature=0
)

sql = response.choices[0].message.content
print(sql)

SELECT Products.ProductName, Suppliers.CompanyName
FROM Products
JOIN Suppliers ON Products.SupplierID = Suppliers.SupplierID
WHERE Products.UnitsInStock = 0;


In [12]:
cursor.execute(sql)
result = cursor.fetchall()
print(result)

[("Chef Anton's Gumbo Mix", 'New Orleans Cajun Delights'), ('Alice Mutton', 'Pavlova, Ltd.'), ('Thüringer Rostbratwurst', 'Plutzer Lebensmittelgroßmärkte AG'), ('Gorgonzola Telino', 'Formaggi Fortini s.r.l.'), ('Perth Pasties', "G'day, Mate")]


In [13]:
prompt = f'''
Given the following question and query result, phrase the answer
in terms that a human can understand.

QUESTION: {question}

RESULT: {result}
'''

messages = [
    { 'role': 'user', 'content': prompt }
]

response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=messages
)

print(response.choices[0].message.content)

The products that are currently out of stock and their respective suppliers are: 
- Chef Anton's Gumbo Mix (Supplier: New Orleans Cajun Delights)
- Alice Mutton (Supplier: Pavlova, Ltd.)
- Thüringer Rostbratwurst (Supplier: Plutzer Lebensmittelgroßmärkte AG)
- Gorgonzola Telino (Supplier: Formaggi Fortini s.r.l.)
- Perth Pasties (Supplier: G'day, Mate)


Show the answer again, but this time use markdown to show tabular results:

In [14]:
from IPython.display import Markdown, display

prompt = f'''
Given the following question and query result, phrase the answer in
terms that a human can understand. Format the results in markdown and
include a markdown table if appropriate.

QUESTION: {question}

RESULT: {result}
'''

messages = [
    { 'role': 'user', 'content': prompt }
]

response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=messages
)

display(Markdown(response.choices[0].message.content))

The following products are currently out of stock:

| Product Name             | Supplier                       |
|--------------------------|--------------------------------|
| Chef Anton's Gumbo Mix   | New Orleans Cajun Delights     |
| Alice Mutton             | Pavlova, Ltd.                  |
| Thüringer Rostbratwurst  | Plutzer Lebensmittelgroßmärkte AG |
| Gorgonzola Telino        | Formaggi Fortini s.r.l.        |
| Perth Pasties            | G'day, Mate                    |

ChatGPT's ability to convert natural-language questions into SQL queries and phrase query results in natural language provides a powerful capability to make information in a database easily discoverable. Tools such as [LangChain](https://www.langchain.com/) provide handy classes for putting LLMs over databases, but this notebook demonstrates how such frameworks work.